# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [18]:
# review the collections in our database
collections = db.list_collection_names()

# Print the collections
print (collections)

['establishments']


In [19]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [20]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
if first_document:
    print("First establishment with a hygiene score of 20:")
    pprint(first_document)
else:
    print("No establishments with a hygiene score of 20 found.")

Number of establishments with a hygiene score of 20: 41
First establishment with a hygiene score of 20:
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6740c0dd5f8636647ffbc914'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'latitude': None,
 'links': [{'href': 'https://ap

In [21]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))

# Display the number of rows in the DataFrame
df = pd.DataFrame(results)
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 41
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  6740c0dd5f8636647ffbc914   110681                  0   
1  6740c0dd5f8636647ffbcc90   612039                  0   
2  6740c0de5f8636647ffbcfa0   730933                  0   
3  6740c0de5f8636647ffbd190   172735                  0   
4  6740c0de5f8636647ffbd19c   172953                  0   
5  6740c0de5f8636647ffbdb3c   512854                  0   
6  6740c0de5f8636647ffbdd5d  1537089                  0   
7  6740c0de5f8636647ffbf280   155648                  0   
8  6740c0de5f8636647ffbf6ca  1012883                  0   
9  6740c0de5f8636647ffbfee3   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [22]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
if first_document:
    print("First establishment in London with RatingValue >= 4:")
    pprint(first_document)
else:
    print("No matching establishments found.")


Number of establishments in London with RatingValue >= 4: 0
No matching establishments found.


In [23]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))

# Display the number of rows in the DataFrame
df = pd.DataFrame(results)
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [24]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.5074  # Example latitude
longitude = -0.1278  # Example longitude

query = {
    "RatingValue": 5,
    "latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
}
sort = [("scores.Hygiene", 1)]  # Sort by hygiene score ascending
limit = 5  # Limit results to 5

# Print the results
results = establishments.find(query).sort(sort).limit(limit)
for result in results:
    pprint(result)


In [25]:
# Convert result to Pandas DataFrame
results = list(establishments.find(query).sort(sort).limit(limit))
df = pd.DataFrame(results)

# Display the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [26]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Run the pipeline and print the number of documents in the result
results = list(establishments.aggregate(pipeline))
print(f"Number of Local Authorities with hygiene score 0: {len(results)}")

# Print the first 10 results
print("Top 10 Local Authorities with hygiene score 0:")
for result in results[:10]:
    pprint(result)


Number of Local Authorities with hygiene score 0: 55
Top 10 Local Authorities with hygiene score 0:
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [27]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 55
First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
